# Monkeypox KG


# Package installation is required only once

In [ ]:
pip install -r packages.txt

In [132]:
import pybel
import pandas as pd
from pybel.dsl import Protein
from pybel.dsl import Abundance
from pybel.dsl import Pathology
from pybel.dsl import BiologicalProcess
from pybel.dsl import Population
from pybel.dsl import Gene
from pybel.dsl import Rna
from pybel.dsl import MicroRna
from pybel.dsl import Fragment
import chembl_webresource_client
import openpyxl
import networkx as nx
from pybel.io.jupyter import to_jupyter
import matplotlib.pyplot as plt
from chembl_webresource_client.new_client import new_client
import pubchempy
import pickle
import re
from ByCovidGraph import *
import pubchempy as pcp
import urllib
from urllib.parse import urlparse
import io

In [33]:
# drugs against monkeypox from https://www.jglobalbiosecurity.com/articles/10.31646/gbio.12

mpox = 'CHEMBL152 CHEMBL203321 CHEMBL1257073 CHEMBL220610 CHEMBL221054 CHEMBL1257073 CHEMBL375768 CHEMBL220150'.split()
mpox

['CHEMBL152',
 'CHEMBL203321',
 'CHEMBL1257073',
 'CHEMBL220610',
 'CHEMBL221054',
 'CHEMBL1257073',
 'CHEMBL375768',
 'CHEMBL220150']

In [34]:
#reading file from pubchem for monkeypox 
#https://pubchem.ncbi.nlm.nih.gov/taxonomy/10244

mpox_pchem = pd.read_csv('data/normalized_data/TaxID_10244_bioactivity.csv')
mpox_pchem_cid = mpox_pchem['cid']
mpox_pchem_cid = list(set(mpox_pchem_cid))
len(mpox_pchem_cid)

24

In [127]:
GetAct = new_client.activity

In [123]:
#Function to retrieve assay ids for compounds
def RetAct(chemblIds,j):
    GetAct = new_client.activity
    ActList = []
    #for i in range(len(chemblIds)):
    #print(chemblIds[0])
    for i in range(len(chemblIds)):
        filtered_list=['assay_chembl_id','assay_type','pchembl_value','target_chembl_id','target_organism','bao_label']
        #acts = GetAct.filter(molecule_chembl_id=chemblIds[i],pchembl_value__isnull=False,assay_type_iregex='(B|F)',target_organism='Homo sapiens').only(filtered_list)
        #acts = GetAct.filter(molecule_chembl_id=chemblIds[i],pchembl_value__isnull=False,target_organism='Homo sapiens').only(filtered_list)
        acts = GetAct.filter(molecule_chembl_id=chemblIds[i],pchembl_value__isnull=False)
        j=j+1

        #acts = [d for d in acts if d.get('target_organism') == 'Homo sapiens']
        #acts = [d for d in acts if d.get('bao_label') == 'single protein format']
        #acts = [d for d in acts if d.get('type') in ['Ki', 'IC50']]
        acts = [d for d in acts if float(d.get('pchembl_value')) >= 6]
        print(j)
        #print(len(acts))
        acts = acts[:5]
        print(acts)
        ActList.append(list(acts))
    #print(ActList)
    named_ActList = dict(zip(chemblIds,ActList))
    named_ActList = {k: v for k, v in named_ActList.items() if v}
    return(named_ActList)


In [36]:
#convert pubchem cids to chembl ids

cid2chembl_dict = []

for id in mpox_pchem_cid:
    #if id != 'None':
    print(id)
    GetChembl = {}
    c=pcp.Compound.from_cid(id)
    #print(id)
    syn = c.synonyms
    for s in syn:
        #if s.startswith('SCHEMBL'):
            #GetChembl['SCHEMBL'] = s
        if s.startswith('CHEMBL'):
            GetChembl['CHEMBL'] = s

    cid2chembl_dict.append(GetChembl)
        

134406
44420748
101879053
101879054
16124688
11243799
155545375
11773602
73124
37542
44341289
44341811
44341812
118895042
118895044
60613
11301322
189926
5271015
11452011
11463022
155521397
45111031
122873


In [37]:
chemblid = []

for id in cid2chembl_dict:
    #chemblid.append()
    #print(id.values())
    chembl = list(id.values())
    #print(chembl[0])
    chemblid.append(chembl[0])
    

In [38]:
chemblid = mpox+chemblid
len(chemblid)

32

In [39]:
chemblid = set(chemblid)
chemblid = list(chemblid)
len(chemblid)

25

In [40]:
pchem_mech = RetMech(chemblid)


[]
[]
[]
[]
[]
[{'mechanism_of_action': 'Envelope phospholipase F13 (p37) inhibitor', 'target_chembl_id': 'CHEMBL4296170'}]
[]
[]
[{'mechanism_of_action': "Inosine-5'-monophosphate dehydrogenase 1 inhibitor", 'target_chembl_id': 'CHEMBL1822'}, {'mechanism_of_action': 'Hepatitis C virus NS5B RNA-dependent RNA polymerase inhibitor', 'target_chembl_id': 'CHEMBL5375'}, {'mechanism_of_action': 'RNA inhibitor', 'target_chembl_id': 'CHEMBL2363073'}]
[{'mechanism_of_action': 'DNA polymerase inhibitor', 'target_chembl_id': 'CHEMBL3988586'}]
[]
[]
[]
[]
[{'mechanism_of_action': 'Human herpesvirus 1 DNA polymerase inhibitor', 'target_chembl_id': 'CHEMBL1872'}]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [41]:
pchem_dis = RetDrugInd(chemblid)

[]
[]
[]
[]
[]
[{'mesh_heading': 'Virus Diseases'}, {'mesh_heading': 'Smallpox'}]
[]
[]
[{'mesh_heading': 'Leukemia, Myeloid, Acute'}, {'mesh_heading': 'Hepatitis C, Chronic'}, {'mesh_heading': 'Hepatitis C'}, {'mesh_heading': 'Hepatitis B, Chronic'}, '...(remaining elements truncated)...']
[{'mesh_heading': 'Cytomegalovirus Infections'}, {'mesh_heading': 'Hemorrhagic Fever, Ebola'}, {'mesh_heading': 'Adenovirus Infections, Human'}, {'mesh_heading': 'Adenoviridae Infections'}, '...(remaining elements truncated)...']
[]
[]
[]
[]
[{'mesh_heading': 'Virus Diseases'}, {'mesh_heading': 'Infections'}, {'mesh_heading': 'Kidney Diseases'}, {'mesh_heading': 'Sarcoma'}, '...(remaining elements truncated)...']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [42]:
pchem_act = RetAct(chemblid,0)

1
[{'activity_comment': None, 'activity_id': 2305823, 'activity_properties': [], 'assay_chembl_id': 'CHEMBL995355', 'assay_description': 'Antiviral activity against Camelpox virus CML14 infected in HEL299 cells assessed as inhibition of virus-induced cytopathic effect after 6 days by Giemsa staining', 'assay_type': 'F', 'assay_variant_accession': None, 'assay_variant_mutation': None, 'bao_endpoint': 'BAO_0000188', 'bao_format': 'BAO_0000218', 'bao_label': 'organism-based format', 'canonical_smiles': 'Nc1ncnc2c1ncn2C[C@H]1COP(=O)(O)CO1', 'data_validity_comment': None, 'data_validity_description': None, 'document_chembl_id': 'CHEMBL1137668', 'document_journal': 'Antimicrob. Agents Chemother.', 'document_year': 2007, 'ligand_efficiency': None, 'molecule_chembl_id': 'CHEMBL112159', 'molecule_pref_name': None, 'parent_molecule_chembl_id': 'CHEMBL112159', 'pchembl_value': '7.00', 'potential_duplicate': 0, 'qudt_units': 'http://www.openphacts.org/units/Nanomolar', 'record_id': 784315, 'relati

5
[{'activity_comment': None, 'activity_id': 14710465, 'activity_properties': [], 'assay_chembl_id': 'CHEMBL3294635', 'assay_description': 'Inhibition of recombinant human AHCY using SAH as substrate assessed as formation of homocysteine after 10 mins', 'assay_type': 'B', 'assay_variant_accession': None, 'assay_variant_mutation': None, 'bao_endpoint': 'BAO_0000190', 'bao_format': 'BAO_0000357', 'bao_label': 'single protein format', 'canonical_smiles': 'Nc1ncc(F)c2c1ncn2[C@@H]1C[C@H](O)[C@@H](O)[C@H]1O', 'data_validity_comment': None, 'data_validity_description': None, 'document_chembl_id': 'CHEMBL3286270', 'document_journal': 'Bioorg. Med. Chem. Lett.', 'document_year': 2014, 'ligand_efficiency': {'bei': '28.41', 'le': '0.55', 'lle': '8.44', 'sei': '6.49'}, 'molecule_chembl_id': 'CHEMBL3290663', 'molecule_pref_name': None, 'parent_molecule_chembl_id': 'CHEMBL3290663', 'pchembl_value': '7.62', 'potential_duplicate': 0, 'qudt_units': 'http://www.openphacts.org/units/Nanomolar', 'record_i

7
[]
8
[]
9
[{'activity_comment': None, 'activity_id': 566848, 'activity_properties': [], 'assay_chembl_id': 'CHEMBL674318', 'assay_description': 'Effective concentration required to inhibit Herpes simplex virus-1(HSV-1) / thymine kinase deficient (TK-)B2006 / VMW1837 induced cytopathicity by 50% in E6SM cell lines', 'assay_type': 'F', 'assay_variant_accession': None, 'assay_variant_mutation': None, 'bao_endpoint': 'BAO_0000188', 'bao_format': 'BAO_0000219', 'bao_label': 'cell-based format', 'canonical_smiles': 'NC(=O)c1ncn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)n1', 'data_validity_comment': None, 'data_validity_description': None, 'document_chembl_id': 'CHEMBL1134563', 'document_journal': 'J. Med. Chem.', 'document_year': 2001, 'ligand_efficiency': None, 'molecule_chembl_id': 'CHEMBL1643', 'molecule_pref_name': 'RIBAVIRIN', 'parent_molecule_chembl_id': 'CHEMBL1643', 'pchembl_value': '8.30', 'potential_duplicate': 0, 'qudt_units': 'http://www.openphacts.org/units/Nanomolar', 'record_id': 17

11
[]
12
[{'activity_comment': None, 'activity_id': 14986435, 'activity_properties': [], 'assay_chembl_id': 'CHEMBL3387285', 'assay_description': 'Inhibition of rabbit erythrocyte SAHase using S-adenosyl-L-homocysteine as substrate assessed as L-homocysteine formation incubated for 5 mins prior to substrate addition measured after 1 hr', 'assay_type': 'B', 'assay_variant_accession': None, 'assay_variant_mutation': None, 'bao_endpoint': 'BAO_0000190', 'bao_format': 'BAO_0000019', 'bao_label': 'assay format', 'canonical_smiles': 'Nc1ncc(F)c2c1ncn2[C@@H]1C[C@H](CCO)[C@@H](O)[C@H]1O', 'data_validity_comment': None, 'data_validity_description': None, 'document_chembl_id': 'CHEMBL3351456', 'document_journal': 'Bioorg. Med. Chem.', 'document_year': 2014, 'ligand_efficiency': None, 'molecule_chembl_id': 'CHEMBL3357216', 'molecule_pref_name': None, 'parent_molecule_chembl_id': 'CHEMBL3357216', 'pchembl_value': '8.05', 'potential_duplicate': 0, 'qudt_units': 'http://www.openphacts.org/units/Nano

16
[{'activity_comment': None, 'activity_id': 13355179, 'activity_properties': [], 'assay_chembl_id': 'CHEMBL2400652', 'assay_description': 'Antiviral activity against Cowpox virus infected in african green monkey Vero 76 cells assessed as inhibition of virus-induced cytopathic effect after 3 days by microscopic analysis', 'assay_type': 'F', 'assay_variant_accession': None, 'assay_variant_mutation': None, 'bao_endpoint': 'BAO_0000190', 'bao_format': 'BAO_0000218', 'bao_label': 'organism-based format', 'canonical_smiles': 'Nc1ncnc2c1ncn2[C@@H]1C[C@H](CCO)[C@@H](O)[C@H]1O', 'data_validity_comment': None, 'data_validity_description': None, 'document_chembl_id': 'CHEMBL2396569', 'document_journal': 'Bioorg. Med. Chem.', 'document_year': 2013, 'ligand_efficiency': None, 'molecule_chembl_id': 'CHEMBL360506', 'molecule_pref_name': None, 'parent_molecule_chembl_id': 'CHEMBL360506', 'pchembl_value': '6.31', 'potential_duplicate': 0, 'qudt_units': 'http://www.openphacts.org/units/Nanomolar', 're

18
[]
19
[{'activity_comment': None, 'activity_id': 14986436, 'activity_properties': [], 'assay_chembl_id': 'CHEMBL3387285', 'assay_description': 'Inhibition of rabbit erythrocyte SAHase using S-adenosyl-L-homocysteine as substrate assessed as L-homocysteine formation incubated for 5 mins prior to substrate addition measured after 1 hr', 'assay_type': 'B', 'assay_variant_accession': None, 'assay_variant_mutation': None, 'bao_endpoint': 'BAO_0000190', 'bao_format': 'BAO_0000019', 'bao_label': 'assay format', 'canonical_smiles': 'Nc1ncc(F)c2c1ncn2[C@@H]1C[C@H](CO)[C@@H](O)[C@H]1O', 'data_validity_comment': None, 'data_validity_description': None, 'document_chembl_id': 'CHEMBL3351456', 'document_journal': 'Bioorg. Med. Chem.', 'document_year': 2014, 'ligand_efficiency': None, 'molecule_chembl_id': 'CHEMBL3357215', 'molecule_pref_name': None, 'parent_molecule_chembl_id': 'CHEMBL3357215', 'pchembl_value': '7.54', 'potential_duplicate': 0, 'qudt_units': 'http://www.openphacts.org/units/Nanom

21
[]
22
[]
23
[{'activity_comment': None, 'activity_id': 1825368, 'activity_properties': [], 'assay_chembl_id': 'CHEMBL918571', 'assay_description': 'Antiviral activity against vaccinia virus IHD-J in BSC40 cells by CPE assay', 'assay_type': 'F', 'assay_variant_accession': None, 'assay_variant_mutation': None, 'bao_endpoint': 'BAO_0000188', 'bao_format': 'BAO_0000218', 'bao_label': 'organism-based format', 'canonical_smiles': 'O=C(NN1C(=O)[C@H]2C3C=CC(C4CC43)[C@H]2C1=O)c1ccncc1', 'data_validity_comment': None, 'data_validity_description': None, 'document_chembl_id': 'CHEMBL1142975', 'document_journal': 'J. Med. Chem.', 'document_year': 2007, 'ligand_efficiency': None, 'molecule_chembl_id': 'CHEMBL220150', 'molecule_pref_name': None, 'parent_molecule_chembl_id': 'CHEMBL220150', 'pchembl_value': '6.30', 'potential_duplicate': 0, 'qudt_units': 'http://www.openphacts.org/units/Nanomolar', 'record_id': 627667, 'relation': '=', 'src_id': 1, 'standard_flag': 1, 'standard_relation': '=', 'sta

25
[{'activity_comment': None, 'activity_id': 1179882, 'activity_properties': [], 'assay_chembl_id': 'CHEMBL800629', 'assay_description': 'Tested for inhibitory concentration against S-adenosyl-homocysteine hydrolase', 'assay_type': 'B', 'assay_variant_accession': None, 'assay_variant_mutation': None, 'bao_endpoint': 'BAO_0000190', 'bao_format': 'BAO_0000357', 'bao_label': 'single protein format', 'canonical_smiles': 'Nc1ncnc2c1ncn2C[C@H](O)CO', 'data_validity_comment': None, 'data_validity_description': None, 'document_chembl_id': 'CHEMBL1127215', 'document_journal': 'Bioorg. Med. Chem. Lett.', 'document_year': 1993, 'ligand_efficiency': {'bei': '31.32', 'le': '0.60', 'lle': '7.79', 'sei': '5.95'}, 'molecule_chembl_id': 'CHEMBL494759', 'molecule_pref_name': '(S)-DHPA', 'parent_molecule_chembl_id': 'CHEMBL494759', 'pchembl_value': '6.55', 'potential_duplicate': 0, 'qudt_units': 'http://www.openphacts.org/units/Nanomolar', 'record_id': 312095, 'relation': '=', 'src_id': 1, 'standard_fla

In [43]:
chemid2 = Ret_chembl_protein(pchem_act) + Ret_chembl_protein(pchem_mech)
chemid2

['CHEMBL613738',
 'CHEMBL613332',
 'CHEMBL614200',
 'CHEMBL613167',
 'CHEMBL612672',
 'CHEMBL340',
 'CHEMBL613495',
 'CHEMBL613493',
 'CHEMBL1293235',
 'CHEMBL614149',
 'CHEMBL3879801',
 'CHEMBL613494',
 'CHEMBL612558',
 'CHEMBL613888',
 'CHEMBL3751646',
 'CHEMBL371',
 'CHEMBL4434621',
 'CHEMBL2664',
 'CHEMBL612545',
 'CHEMBL613168',
 'CHEMBL276',
 'CHEMBL613120',
 'CHEMBL3988586',
 'CHEMBL5375',
 'CHEMBL4296170',
 'CHEMBL1872',
 'CHEMBL2363073',
 'CHEMBL1822']

In [45]:
chem2uni = chembl2uniprot(chemid2,0)
chem2uni

old 10
newLen 8
2
[{'xref_id': 'R-HSA-211981', 'xref_name': 'Xenobiotics.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-5423646', 'xref_name': 'Aflatoxin activation and detoxification.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-9027307', 'xref_name': 'Biosynthesis of maresin-like SPMs.', 'xref_src_db': 'Reactome'}, {'component_synonym': 'CYP3A4', 'syn_type': 'GENE_SYMBOL'}]
[{'xref_id': 'R-HSA-1221632', 'xref_name': 'Meiotic synapsis. [P02545-2]', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-2980766', 'xref_name': 'Nuclear Envelope Breakdown.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-2995383', 'xref_name': 'Initiation of Nuclear Envelope (NE) Reformation.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-352238', 'xref_name': 'Breakdown of the nuclear lamina. [P02545-1]', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-381038', 'xref_name': 'XBP1(S) activates chaperone genes.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-4419969', 'xref_name': 'Depolymerisation of th

{'CHEMBL340': [{'xref_id': 'R-HSA-211981',
   'xref_name': 'Xenobiotics.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-5423646',
   'xref_name': 'Aflatoxin activation and detoxification.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-9027307',
   'xref_name': 'Biosynthesis of maresin-like SPMs.',
   'xref_src_db': 'Reactome'},
  {'component_synonym': 'CYP3A4', 'syn_type': 'GENE_SYMBOL'}],
 'CHEMBL1293235': [{'xref_id': 'R-HSA-1221632',
   'xref_name': 'Meiotic synapsis. [P02545-2]',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-2980766',
   'xref_name': 'Nuclear Envelope Breakdown.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-2995383',
   'xref_name': 'Initiation of Nuclear Envelope (NE) Reformation.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-352238',
   'xref_name': 'Breakdown of the nuclear lamina. [P02545-1]',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-381038',
   'xref_name': 'XBP1(S) activates chaperone genes.',
   'xref_src_db': '

In [46]:
pchem_act_new= chembl2gene2path(chem2uni,pchem_act)
pchem_act_new

{'CHEMBL112159': [{'activity_comment': None,
   'activity_id': 2305823,
   'activity_properties': [],
   'assay_chembl_id': 'CHEMBL995355',
   'assay_description': 'Antiviral activity against Camelpox virus CML14 infected in HEL299 cells assessed as inhibition of virus-induced cytopathic effect after 6 days by Giemsa staining',
   'assay_type': 'F',
   'assay_variant_accession': None,
   'assay_variant_mutation': None,
   'bao_endpoint': 'BAO_0000188',
   'bao_format': 'BAO_0000218',
   'bao_label': 'organism-based format',
   'canonical_smiles': 'Nc1ncnc2c1ncn2C[C@H]1COP(=O)(O)CO1',
   'data_validity_comment': None,
   'data_validity_description': None,
   'document_chembl_id': 'CHEMBL1137668',
   'document_journal': 'Antimicrob. Agents Chemother.',
   'document_year': 2007,
   'ligand_efficiency': None,
   'molecule_chembl_id': 'CHEMBL112159',
   'molecule_pref_name': None,
   'parent_molecule_chembl_id': 'CHEMBL112159',
   'pchembl_value': '7.00',
   'potential_duplicate': 0,
   'qu

In [20]:
mpox_graph = pybel.BELGraph(name='Monkeypox Graph')

In [47]:
mpox_graph = chem2moa_rel_2(pchem_mech,mpox_graph)
mpox_graph = chem2dis_rel(pchem_dis,mpox_graph)
mpox_graph = chem2act_rel_2(pchem_act_new,mpox_graph)
mpox_graph = chem2gene2path_rel(chem2uni,mpox_graph)
#to_jupyter(mpox_graph)

In [48]:
mpox_graph.summarize()

---------------------  ---------------
Name                   Monkeypox Graph
Version
Number of Nodes        280
Number of Namespaces   9
Number of Edges        618
Number of Annotations  0
Number of Citations    2
Number of Authors      0
Number of Components   13
Number of Warnings     0
Network Density        7.91E-03
---------------------  ---------------

Type (7)             Count  Example
-----------------  -------  -----------------------------------------------------------------------
miRNA                   80  m(HP:CHEMBL613332)
BiologicalProcess       53  bp(MOA:"Hepatitis C virus NS5B RNA-dependent RNA polymerase inhibitor")
Abundance               41  a(ChEMBLAssay:CHEMBL918575)
Population              36  pop(Disease:"Uterine Cervical Dysplasia")
RNA                     30  r(Pathway:"Breakdown of the nuclear lamina. [P02545-1]")
Protein                 23  p(MP:CYB5R3)
Pathology               17  path(ChEMBL:CHEMBL375768)

Prefix (9)    Name                            C

# Working with monkeypox proteins from uniprot


In [26]:
mpox_prot_df = pd.read_excel('data/normalized_data/uniprot-taxonomy Monkeypox+virus+[10244] -filtered-reviewed yes.xlsx')

C:\Users\reagon.karki\Anaconda3\envs\bycovid_new\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [27]:
mpox_prot= list(mpox_prot_df['Entry'])
mpox_prot

['Q8V4Y0',
 'Q8V4S4',
 'Q8V571',
 'P04363',
 'Q8V4V3',
 'Q8V4U9',
 'Q8V4T3',
 'Q8V518',
 'Q8V4V4',
 'Q8V4T7',
 'Q8V566']

In [28]:
mpox_prot_ext = ExtractFromUniProt(mpox_prot)

https://www.uniprot.org/uniprot/Q8V4Y0.txt
Q8V4Y0
https://www.uniprot.org/uniprot/Q8V4S4.txt
Q8V4S4
https://www.uniprot.org/uniprot/Q8V571.txt
Q8V571
https://www.uniprot.org/uniprot/P04363.txt
P04363
https://www.uniprot.org/uniprot/Q8V4V3.txt
Q8V4V3
https://www.uniprot.org/uniprot/Q8V4U9.txt
Q8V4U9
https://www.uniprot.org/uniprot/Q8V4T3.txt
Q8V4T3
https://www.uniprot.org/uniprot/Q8V518.txt
Q8V518
https://www.uniprot.org/uniprot/Q8V4V4.txt
Q8V4V4
https://www.uniprot.org/uniprot/Q8V4T7.txt
Q8V4T7
https://www.uniprot.org/uniprot/Q8V566.txt
Q8V566


In [30]:
mpox_prot_ext

{'Q8V4Y0': {'Disease': {},
  'Reactome': {},
  'Function': {'carbonate dehydratase activity': 'GO:0004089',
   'zinc ion binding': 'GO:0008270'},
  'BioProcess': {'viral entry into host cell': 'GO:0046718',
   'virion attachment to host cell': 'GO:0019062'},
  'Gene': nan,
  'Length': 304,
  'Mass': 35278,
  'Sequence': 'MPQQLSPINIETKKAISDTRLKTLDIHYNESKPTTIQNTGKLVRINFKGGYISGGFLPNEYVLSTIHIYWGKEDDYGSNHLIDVYKYSGEINLVHWNKKKYSSYEEAKKHDDGIIIIAIFLQVSDHKNVYFQKIVNQLDSIRSANMSAPFDSVFYLDNLLPSTLDYFTYLGTTINHSADAAWIIFPTPINIHSDQLSKFRTLLSSSNHEGKPHYITENYRNPYKLNDDTQVYYSGEIIRAATTSPVRENYFMKWLSDLREACFSYYQKYIEGNKTFAIIAIVFVFILTAILFLMSQRYSREKQN'},
 'Q8V4S4': {'Disease': {},
  'Reactome': {},
  'Function': {"2',3'-cyclic GMP-AMP binding": 'GO:0061507',
   'nuclease activity': 'GO:0004518'},
  'BioProcess': {'suppression by virus of host innate immune response': 'GO:0039503'},
  'Gene': 'B4R',
  'Length': 503,
  'Mass': 57454,
  'Sequence': 'MFYAHAFGGYDENLHAFPRISSTVANDVRKYSVVSVYNKKYNIVKNKYMWCNSQVNKRYIGALLPMFECNE

In [28]:
str(mpox_prot_ext['Q8V4S4']['Gene'])

'B4R'

In [22]:

def uniprot_rel(named_uprotList,itmpGraph):
    
    for item in named_uprotList:
        #print(named_uprotList[item]['Function'].keys())
        fun=list(named_uprotList[item]['Function'].keys())
        bp = list(named_uprotList[item]['BioProcess'].keys())
        for f in fun:
            if str(named_uprotList[item]['Gene']) != 'nan':
                itmpGraph.add_association(Protein(namespace='MP',name=named_uprotList[item]['Gene']),MicroRna(namespace='Function',name=f),
                                         citation='UniProt database',evidence='UniProt query')
            else:
                itmpGraph.add_association(Protein(namespace='MP',name=item),MicroRna(namespace='Function',name=f),
                                         citation='UniProt database',evidence='UniProt query')
                
        
        for b in bp:
            if str(named_uprotList[item]['Gene']) != 'nan':
                itmpGraph.add_association(Protein(namespace='MP',name=named_uprotList[item]['Gene']),BiologicalProcess(namespace='BioPro',name=b),
                                         citation='UniProt database',evidence='UniProt query')
            else:
                itmpGraph.add_association(Protein(namespace='MP',name=item),MicroRna(namespace='Function',name=b),
                                         citation='UniProt database',evidence='UniProt query')
        
    return(itmpGraph)
        

In [23]:
#mpox_graph = pybel.BELGraph(name='Monkeypox Graph')

In [49]:
mpox_graph = uniprot_rel(mpox_prot_ext,mpox_graph)

In [50]:
mpox_graph = uniprot_rel(mcards_prot,mpox_graph)

In [53]:
mpox_graph.summarize

Name,Monkeypox Graph
Version,
Nodes,280
Namespaces,9
Edges,618
Annotations,0
Citations,2
Authors,0
Components,13
Warnings,0
Network Density,0.00791


In [54]:
to_jupyter(mpox_graph)

<IPython.core.display.Javascript object>

# Genes from Malacards


In [4]:
mpox_genes = 'IFNA1 C4A C4B CCL26 KLRC3 CD4 CD8A OR10G6 LINCO1285 KLRK1 CD55 CYB5R3 CD46 IRF3'.split(" ")
mpox_genes

['IFNA1',
 'C4A',
 'C4B',
 'CCL26',
 'KLRC3',
 'CD4',
 'CD8A',
 'OR10G6',
 'LINCO1285',
 'KLRK1',
 'CD55',
 'CYB5R3',
 'CD46',
 'IRF3']

In [6]:
mpox_genes = pd.read_excel('data/normalized_data/malacardsGenes.xlsx')

In [7]:
mpox_genes.columns

Index(['yourlist:M20220620A084FC58F6BBA219896F365D15F2EB44516BBDJ', 'Entry',
       'Entry name', 'Status', 'Protein names', 'Gene names', 'Organism',
       'Gene names  (primary )'],
      dtype='object')

In [95]:
mpox_prot = list(mpox_genes['Entry'])
mpox_prot

['P0C0L4',
 'P0C0L5',
 'Q9Y258',
 'Q07444',
 'P01730',
 'P01732',
 'Q8NH81',
 'P26718',
 'P08174',
 'P00387',
 'P15529',
 'Q14653',
 'P01562']

In [11]:
mcards_prot = ExtractFromUniProt(mpox_prot)

https://www.uniprot.org/uniprot/P0C0L4.txt
P0C0L4
https://www.uniprot.org/uniprot/P0C0L5.txt
P0C0L5
https://www.uniprot.org/uniprot/Q9Y258.txt
Q9Y258
https://www.uniprot.org/uniprot/Q07444.txt
Q07444
https://www.uniprot.org/uniprot/P01730.txt
P01730
https://www.uniprot.org/uniprot/P01732.txt
P01732
https://www.uniprot.org/uniprot/Q8NH81.txt
Q8NH81
https://www.uniprot.org/uniprot/P26718.txt
P26718
https://www.uniprot.org/uniprot/P08174.txt
P08174
https://www.uniprot.org/uniprot/P00387.txt
P00387
https://www.uniprot.org/uniprot/P15529.txt
P15529
https://www.uniprot.org/uniprot/Q14653.txt
Q14653
https://www.uniprot.org/uniprot/P01562.txt
P01562


In [12]:
mcards_prot

{'P0C0L4': {'Disease': {'Complement component 4A deficiency (C4AD)': '614380',
   'Systemic lupus erythematosus (SLE)': '152700'},
  'Reactome': {'Initial triggering of complement': 'R-HSA-166663',
   'Activation of C3 and C5': 'R-HSA-174577',
   'Regulation of Insulin-like Growth Factor (IGF) transport and uptake by Insulin-like Growth Factor Binding Proteins (IGFBPs)': 'R-HSA-381426',
   'Post-translational protein phosphorylation': 'R-HSA-8957275',
   'Regulation of Complement cascade': 'R-HSA-977606'},
  'Function': {'complement component C1q complex binding': 'GO:0001849',
   'endopeptidase inhibitor activity': 'GO:0004866'},
  'BioProcess': {'complement activation': 'GO:0006956',
   'complement activation, classical pathway': 'GO:0006958',
   'inflammatory response': 'GO:0006954',
   'innate immune response': 'GO:0045087',
   'positive regulation of apoptotic cell clearance': 'GO:2000427'},
  'Gene': 'C4A',
  'Length': 1744,
  'Mass': 192785,
  'Sequence': 'MRLLWGLIWASSFFTLSLQKPR

In [55]:
#Exporting the graph

#to cytoscape compatible graphml 
pybel.to_graphml(mpox_graph,'data/normalized_data/Monkeypox_KG.graphml')

#to regular BEL format
pybel.dump(mpox_graph,'data/normalized_data/Monkeypox_KG.bel')

#to neo4j
pybel.to_csv(mpox_graph,'data/normalized_data/Monkeypox_KG.csv')

#to sif
pybel.to_sif(mpox_graph,'data/normalized_data/Monkeypox_KG.sif')

# Convert uniprot to chembl

In [110]:
from chembl_webresource_client.new_client import new_client

target = new_client.target
gene_name = 'P01730'
res = target.filter(target_synonym__icontains=gene_name)
for i in res:
    print(i)

In [105]:
query = ' '.join(mpox_prot)
query

'P0C0L4 P0C0L5 Q9Y258 Q07444 P01730 P01732 Q8NH81 P26718 P08174 P00387 P15529 Q14653 P01562'

In [108]:
import urllib.parse
import urllib.request

url = 'https://www.uniprot.org/uploadlists/'

query = 'P0C0L4 P0C0L5 Q9Y258 Q07444 P01730 P01732 Q8NH81 P26718 P08174 P00387 P15529 Q14653 P01562'
params = {
'from': 'ACC+ID',
'to': 'CHEMBL_ID',
'format': 'tab',
'query': query
}

data = urllib.parse.urlencode(params)
data = data.encode('utf-8')
req = urllib.request.Request(url, data)
with urllib.request.urlopen(req) as f:
   response = f.read()
print(response.decode('utf-8'))

b'From\tTo\nP01730\tCHEMBL2754\nP00387\tCHEMBL2146\nQ14653\tCHEMBL4523293\nP01562\tCHEMBL3713007\n'
From	To
P01730	CHEMBL2754
P00387	CHEMBL2146
Q14653	CHEMBL4523293
P01562	CHEMBL3713007



In [106]:
prot2chembl =

b'From\tTo\nP01730\tCHEMBL2754\nP00387\tCHEMBL2146\nQ14653\tCHEMBL4523293\nP01562\tCHEMBL3713007\n'

In [100]:
query = ' '.join(mpox_prot)
query

'P0C0L4 P0C0L5 Q9Y258 Q07444 P01730 P01732 Q8NH81 P26718 P08174 P00387 P15529 Q14653 P01562'

In [96]:
mpox_prot

['P0C0L4',
 'P0C0L5',
 'Q9Y258',
 'Q07444',
 'P01730',
 'P01732',
 'Q8NH81',
 'P26718',
 'P08174',
 'P00387',
 'P15529',
 'Q14653',
 'P01562']

In [136]:
targets_api  = new_client.target


In [176]:
# Get target information from ChEMBL but restrict it to specified values only

#uniprot_id = "P01562"
targets = targets_api.get(target_components__accession="Q14653").only(
    "target_chembl_id", "organism", "pref_name", "target_type"
)

targets[0]

{'organism': 'Homo sapiens',
 'pref_name': 'Interferon regulatory factor 3',
 'target_chembl_id': 'CHEMBL4523293',
 'target_type': 'SINGLE PROTEIN'}

In [169]:
mpox_prot2chembl = []
for item in mpox_prot:
    targets = targets_api.get(target_components__accession=item).only(
    "target_chembl_id", "organism", "pref_name", "target_type")
    mpox_prot2chembl.append(targets[0])

In [171]:
mpox_prot2chembl

[None,
 None,
 None,
 None,
 {'organism': 'Homo sapiens',
  'pref_name': 'T-cell surface antigen CD4',
  'target_chembl_id': 'CHEMBL2754',
  'target_type': 'SINGLE PROTEIN'},
 None,
 None,
 None,
 None,
 {'organism': 'Homo sapiens',
  'pref_name': 'NADH-cytochrome b5 reductase',
  'target_chembl_id': 'CHEMBL2146',
  'target_type': 'SINGLE PROTEIN'},
 None,
 {'organism': 'Homo sapiens',
  'pref_name': 'Interferon regulatory factor 3',
  'target_chembl_id': 'CHEMBL4523293',
  'target_type': 'SINGLE PROTEIN'},
 {'organism': 'Homo sapiens',
  'pref_name': 'Interferon alpha-1/13',
  'target_chembl_id': 'CHEMBL3713007',
  'target_type': 'SINGLE PROTEIN'}]

In [172]:
named_mpox_prot2chembl = dict(zip(mpox_prot,mpox_prot2chembl))

In [173]:
named_mpox_prot2chembl = {k: v for k, v in named_mpox_prot2chembl.items() if v}

In [177]:
named_mpox_prot2chembl

{'P01730': {'organism': 'Homo sapiens',
  'pref_name': 'T-cell surface antigen CD4',
  'target_chembl_id': 'CHEMBL2754',
  'target_type': 'SINGLE PROTEIN'},
 'P00387': {'organism': 'Homo sapiens',
  'pref_name': 'NADH-cytochrome b5 reductase',
  'target_chembl_id': 'CHEMBL2146',
  'target_type': 'SINGLE PROTEIN'},
 'Q14653': {'organism': 'Homo sapiens',
  'pref_name': 'Interferon regulatory factor 3',
  'target_chembl_id': 'CHEMBL4523293',
  'target_type': 'SINGLE PROTEIN'},
 'P01562': {'organism': 'Homo sapiens',
  'pref_name': 'Interferon alpha-1/13',
  'target_chembl_id': 'CHEMBL3713007',
  'target_type': 'SINGLE PROTEIN'}}

In [184]:
getchem = []
for item in named_mpox_prot2chembl:
    getchem.append(named_mpox_prot2chembl[item]['target_chembl_id'])
getchem

['CHEMBL2754', 'CHEMBL2146', 'CHEMBL4523293', 'CHEMBL3713007']

In [187]:
RetAct(getchem,0)

1
[]
2
[]
3
[]
4
[]


{}

In [191]:
bioactivities = bioactivities_api.filter(
    target_chembl_id='CHEMBL3713007', type="IC50", relation="=", assay_type="B"
).only(
    "activity_id",
    "assay_chembl_id",
    "assay_description",
    "assay_type",
    "molecule_chembl_id",
    "type",
    "standard_units",
    "relation",
    "standard_value",
    "target_chembl_id",
    "target_organism",
)

In [192]:
bioactivities

[{'organism': 'Homo sapiens', 'target_chembl_id': 'CHEMBL3713007'}]

In [193]:
test = chembl2uniprot(getchem,0)

old 4
newLen 4
0
[{'xref_id': 'R-HSA-1462054', 'xref_name': 'Alpha-defensins.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-167590', 'xref_name': 'Nef Mediated CD4 Down-regulation.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-173107', 'xref_name': 'Binding and entry of HIV virion.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-180534', 'xref_name': 'Vpu mediated degradation of CD4.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-202424', 'xref_name': 'Downstream TCR signaling.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-202427', 'xref_name': 'Phosphorylation of CD3 and TCR zeta chains.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-202430', 'xref_name': 'Translocation of ZAP-70 to Immunological synapse.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-202433', 'xref_name': 'Generation of second messenger molecules.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-389948', 'xref_name': 'PD-1 signaling.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-449836', 'xref_name